# Pipeline

1. Preprocessing
2. Data Extraction
3. **Model**

This file initializes the model and makes predictions too.

# Imports

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [2]:
# Get feature dataframe from pickle file
df = pd.read_pickle('feature_frame.pkl')

In [3]:
df.head(10)

,HomeID,AwayID,FTHG,FTAG,FTR,H_WIN_PCT_home,H_DRAW_PCT_home,A_WIN_PCT_home,A_DRAW_PCT_home,H_WIN_PCT_away,...,REST_away,H_GS_AVG_league,A_GS_AVG_league,H_GS_STD_league,A_GS_STD_league,H_WIN_PCT_league,DRAW_PCT_league,TEAM_CNT_league,GD_STD_league,RND_CNT_league
0,0,15,3,0,1,0.000000,0.000000,0.0,0.333333,0.000000,...,7,1.600000,0.900000,0.968468,0.758856,0.566667,0.300000,20,0.922889,38
1,1,12,1,0,1,1.000000,0.000000,0.0,0.000000,0.000000,...,7,1.600000,0.900000,0.968468,0.758856,0.566667,0.300000,20,0.922889,38
2,2,16,0,0,0,1.000000,0.000000,0.0,0.500000,0.000000,...,7,1.600000,0.900000,0.968468,0.758856,0.566667,0.300000,20,0.922889,38
3,3,14,6,0,1,0.500000,0.500000,0.0,0.000000,0.333333,...,7,1.600000,0.900000,0.968468,0.758856,0.566667,0.300000,20,0.922889,38
4,4,11,2,2,0,0.666667,0.333333,0.0,0.000000,0.000000,...,7,1.600000,0.900000,0.968468,0.758856,0.566667,0.300000,20,0.922889,38
5,5,18,0,0,0,1.000000,0.000000,0.0,0.500000,1.000000,...,7,1.600000,0.900000,0.968468,0.758856,0.566667,0.300000,20,0.922889,38
6,6,19,0,4,2,0.500000,0.500000,0.0,0.000000,0.500000,...,7,1.600000,0.900000,0.968468,0.758856,0.566667,0.300000,20,0.922889,38
7,7,13,2,1,1,0.000000,0.000000,0.0,0.000000,1.000000,...,7,1.600000,0.900000,0.968468,0.758856,0.566667,0.300000,20,0.922889,38
8,8,10,1,1,0,0.666667,0.000000,0.0,0.000000,0.500000,...,7,1.631579,0.894737,1.239465,0.923843,0.552632,0.315789,20,1.308686,38
9,9,17,3,0,1,1.000000,0.000000,0.0,1.000000,0.000000,...,7,1.615385,0.897436,1.227222,0.911762,0.538462,0.333333,20,1.307097,38


# Extracting target variables and features

In [4]:
# If True, the goal difference for a specific game is seen as the target variable.
# (e.g. -3 for a game outcome of 1:4, or 2 for 3:1) .
#
# If False, we just want to predict the winner.
# 1 = Home team wins, 0 = Draw, 2 = Away team wins
predict_goal_difference = False

if predict_goal_difference:
    y = df['FTHG'] - df['FTAG']
else:
    y = df['FTR']

X = df.iloc[:,5:]    # Remove unnecessary columns (IDs etc.) from features

# Ranked Probability Score (RPS)

In [26]:
def rps(pred, actual_value, r=3):
    '''Returns the ranked probability score for a single given game.
    (see Hubacek paper for formula)
    
    Arguments:
    pred -- predicted results; in vector form (e.g. [0.1, 0.6, 0.3])
    actual_value -- actual result (0, 1 or 2); not in vector form yet
    r -- number of categories (3 for football)
    '''
    value_vec = [0, 0, 0]
    
    # Bring value_vec into 1, 0, 2 order
    if actual_value == 0:
        value_vec[1] = 1
    elif actual_value == 2:
        value_vec[0] == 1
    elif actual_value == 1:
        value_vec[2] == 1
    else:
        raise Exception('Prediction was not in [1, 0, 2].')
    #value_vec = [0, 0, 1]
    
    #pred[0], pred[1] = pred[1], pred[0]
    
    pred[0], pred[1], pred[2] = pred[2], pred[0], pred[1]   # order: loss, draw, win
    
    #print(pred)
    
    rps = 0
    
    for i in range(0, r-1):    # r-1 becomes r because of the exclusion of range()
        bracket_part = 0
        for j in range(0, i+1):    # same for i and i+1
            bracket_part += pred[j] - value_vec[j]
        
        rps += np.square(bracket_part)
    
    rps *= 1 / (r - 1)
    
    return rps

# First model fit

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.222, random_state=16)

In [7]:
# n_estimators=5 seems to be the best choice for now
xgb_cl = xgb.XGBClassifier(objective='multi:softprob', n_estimators=5, seed=16)

In [8]:
xgb_cl.fit(X_train, y_train)

d:\python38\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:09:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=5, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=16, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=16, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

# Evaluation

In [9]:
preds = xgb_cl.predict(X_test)
accuracy = float(np.sum(preds == y_test))/y_test.shape[0]

print(f'accuracy: {accuracy}')

accuracy: 0.5268258913118726


In [24]:
# Predictions in form of [0.2, 0.5, 0.3] in the order of draw, home win, away win.
# This is because it gets ordered like 0, 1, 2.
proba_preds = xgb_cl.predict_proba(X_test)

# List of RPS scores for every game in the test set.
rps_list = [rps(pred, y_test.iloc[i]) for i, pred in enumerate(proba_preds)]

# Average ranked probability score.
np.mean(rps_list)

0.20745323990626627

In [12]:
# Looking at train set accuracy to get an intuition of how much the model overfits
preds = xgb_cl.predict(X_train)
accuracy = float(np.sum(preds == y_train))/y_train.shape[0]

print(f'accuracy on train set: {accuracy}')

accuracy on train set: 0.5905364022522968
